# 국토교통부_오피스텔 전월세 신고 조회 서비스


- 데이터 출처: https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15059249
- 지역코드 출처: 주민등록주소코드 변경내역(2018. 1. 22. 현재)    https://www.mois.go.kr/frt/bbs/type001/commonSelectBoardArticle.do?bbsId=BBSMSTR_000000000052&nttId=61552 내 첨부파일 jscode20180122.zip   
주의: 일일 트래픽 1000회 제한

In [3]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen

## 함수 설정

In [4]:
def collect_offi_rent(ym, lawd_cd) :
    
    # 제공받은 API 
    API_KEY = 'bHg%2FpafJkgCqoGEFC3cI5IvynqkFuwK53BYhBu3bqMz7I%2BX%2BEYH8R9jV%2FCMc06dm11DO%2FksvWzs5sIUeyCjiTQ%3D%3D'
    
    # URL
    url = 'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcOffiRent'
 
    # 요청메시지
    url = url + '?serviceKey=' + API_KEY + '&LAWD_CD=' + lawd_cd + '&DEAL_YMD=' + ym 
 
    resultXML = urlopen(url)
    result = resultXML.read()
    xmlsoup = BeautifulSoup(result, 'lxml-xml')
 
    te = xmlsoup.findAll('item')
 
    # 데이터프레임 결과물
    offi_rent = pd.DataFrame()
 
    for t in te:
        deposit = t.find('보증금').text
        mon_rent = t.find('월세').text

        try:
            build_y = t.find('건축년도').text
        except:
            build_y = "" 

        deal_y = t.find('년').text
        deal_m = t.find('월').text
        deal_day = t.find('일').text
        dong = t.find('법정동').text
        apt_nm = t.find('단지').text
        area_exc = t.find('전용면적').text
        
        try:
            region_num = t.find("지번").text
        except:
            region_num = ""

        region_code = t.find('지역코드').text
        floor = t.find('층').text
 
        temp = pd.DataFrame(([[deposit, mon_rent, build_y, deal_y, deal_m, deal_day, dong, apt_nm, area_exc, region_num, region_code, floor]]),
                           columns = ['deposit', 'mon_rent', 'build_y', 'deal_y', 'deal_m', 'deal_day', 'dong', 'apt_nm', 'area_exc', 'region_num', 'region_code', 'floor'])
        offi_rent = pd.concat([offi_rent, temp])
 
    offi_rent = offi_rent.reset_index(drop=True)
 
    return offi_rent

## 수집

#### 자료 수집 현황
2020년 data => 7/19 완료    

### **** 2019년부터 진행 예정 ****

In [6]:
if __name__ == "__main__":

    # 지역코드 가져오기
    code = pd.read_excel("KIKcd_B.20180122.xlsx")
    code_cap = code[(code["시도명"]=="서울특별시") | (code["시도명"] == "경기도")]
    code_cap = code_cap[code_cap["읍면동명"].isnull() == True]
    code_cap = code_cap[code_cap["시군구명"].isnull() == False]
    code_cap["region_code"] = code_cap["법정동코드"].astype(str).str[0:5]
    code_cap = code_cap.reset_index(drop=True)
    offi_rent = pd.DataFrame()

    # 기간설정(연월: 'yyyymm')
    ym = list(['202001', '202002', '202003', '202004', '202005', '202006', 
               '202007', '202008', '202009', '202010', '202011','202012'])
 
    for m in ym:
        i = 0
        for co in code_cap['region_code'] :
            temp = collect_offi_rent(m, co)
            temp['sido'] = code_cap['시도명'][i]
            temp['sigungu'] = code_cap['시군구명'][i]
            offi_rent = pd.concat([offi_rent, temp])
            i += 1
            print(co + ", " + str(len(temp)) + "개 완료")
            
        print("******** " + str(m) + " 완료 ********")

11110, 64개 완료
11140, 82개 완료
11170, 82개 완료
11200, 115개 완료
11215, 106개 완료
11230, 324개 완료
11260, 59개 완료
11290, 41개 완료
11305, 9개 완료
11320, 63개 완료
11350, 37개 완료
11380, 267개 완료
11410, 136개 완료
11440, 471개 완료
11470, 92개 완료
11500, 812개 완료
11530, 256개 완료
11545, 193개 완료
11560, 608개 완료
11590, 39개 완료
11620, 308개 완료
11650, 164개 완료
11680, 288개 완료
11710, 417개 완료
11740, 150개 완료
41110, 0개 완료
41111, 6개 완료
41113, 72개 완료
41115, 79개 완료
41117, 273개 완료
41130, 0개 완료
41131, 70개 완료
41133, 134개 완료
41135, 401개 완료
41150, 71개 완료
41170, 0개 완료
41171, 27개 완료
41173, 113개 완료
41190, 265개 완료
41210, 108개 완료
41220, 16개 완료
41250, 3개 완료
41270, 0개 완료
41271, 14개 완료
41273, 254개 완료
41280, 0개 완료
41281, 87개 완료
41285, 245개 완료
41287, 68개 완료
41290, 0개 완료
41310, 52개 완료
41360, 143개 완료
41370, 58개 완료
41390, 53개 완료
41410, 76개 완료
41430, 32개 완료
41450, 550개 완료
41460, 0개 완료
41461, 22개 완료
41463, 93개 완료
41465, 128개 완료
41480, 71개 완료
41500, 48개 완료
41550, 7개 완료
41570, 129개 완료
41590, 308개 완료
41610, 0개 완료
41630, 2개 완료
41650, 0개 완료
41670, 4개 완료
41800, 

41430, 12개 완료
41450, 572개 완료
41460, 0개 완료
41461, 51개 완료
41463, 46개 완료
41465, 99개 완료
41480, 103개 완료
41500, 25개 완료
41550, 5개 완료
41570, 189개 완료
41590, 355개 완료
41610, 2개 완료
41630, 3개 완료
41650, 1개 완료
41670, 1개 완료
41800, 0개 완료
41820, 1개 완료
41830, 0개 완료
******** 202008 완료 ********
11110, 79개 완료
11140, 77개 완료
11170, 59개 완료
11200, 99개 완료
11215, 64개 완료
11230, 142개 완료
11260, 82개 완료
11290, 24개 완료
11305, 13개 완료
11320, 70개 완료
11350, 18개 완료
11380, 150개 완료
11410, 43개 완료
11440, 286개 완료
11470, 84개 완료
11500, 729개 완료
11530, 228개 완료
11545, 235개 완료
11560, 460개 완료
11590, 45개 완료
11620, 156개 완료
11650, 155개 완료
11680, 215개 완료
11710, 225개 완료
11740, 117개 완료
41110, 0개 완료
41111, 3개 완료
41113, 58개 완료
41115, 54개 완료
41117, 132개 완료
41130, 0개 완료
41131, 26개 완료
41133, 122개 완료
41135, 230개 완료
41150, 67개 완료
41170, 0개 완료
41171, 34개 완료
41173, 69개 완료
41190, 203개 완료
41210, 80개 완료
41220, 22개 완료
41250, 2개 완료
41270, 0개 완료
41271, 23개 완료
41273, 155개 완료
41280, 0개 완료
41281, 185개 완료
41285, 202개 완료
41287, 66개 완료
41290, 0개 완료
41310, 41개 완료


In [7]:
offi_rent

,deposit,mon_rent,build_y,deal_y,deal_m,deal_day,dong,apt_nm,area_exc,region_num,region_code,floor,sido,sigungu
0,"15,000",0,1998,2020,1,21,필운동,파크뷰타워,23.24,285-5,11110,7,서울특별시,종로구
1,"1,000",85,2004,2020,1,1,내수동,경희궁의 아침 2단지,32.43,71,11110,16,서울특별시,종로구
2,"22,000",70,2004,2020,1,3,내수동,광화문시대,56.56,74,11110,13,서울특별시,종로구
3,"21,000",0,2004,2020,1,3,내수동,용비어천가,33,75,11110,13,서울특별시,종로구
4,"13,000",75,2004,2020,1,10,내수동,경희궁의 아침 4단지,56.48,73,11110,16,서울특별시,종로구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,"2,000",50,2020,2020,12,14,덕계동,주건축물제1동,82.29,691-1,41630,4,경기도,양주시
0,"5,500",0,2013,2020,12,6,현암동,도형타운 비동,49.38,187-29,41670,5,경기도,여주시
1,"2,000",60,2017,2020,12,30,현암동,애플하우스 D,84.98,189-18,41670,6,경기도,여주시
0,500,35,2012,2020,12,25,전곡읍 전곡리,전곡쉐르빌,18.97,299-7,41800,2,경기도,연천군


In [8]:
offi_rent.to_csv('offi_rent_2020.csv', index=False)